In [8]:
import jieba
import time
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer  
from scipy import io
import json
from joblib import load   
import csv

In [7]:
clf=load('../../save_model/svm_model_linear.joblib')

In [21]:
class MessageCountVectorizer(sklearn.feature_extraction.text.CountVectorizer):
    def build_analyzer(self):
        def analyzer(doc):
            words = jieba.cut(doc)
            return words
        return analyzer

In [22]:
import pandas as pd 
vect = 	MessageCountVectorizer(max_df=0.9,min_df=2)
df=pd.read_csv('../data/sample3.csv')
train_content=df['content']
vect.fit(train_content)
df=pd.read_csv('../data/test.csv')
test_content=df['content']
test_content=vect.transform(test_content)


In [23]:
# test_content = io.mmread('../data/test_content_vector.mtx')
# label=[]
# with open('../data/test_label_vector.json', 'r') as f:
#     label = json.load(f)
label_t=[]
with open('../data/test.csv','r') as f:
    re=csv.reader(f)
    next(re)
    for row in re:
        label_t.append(int(row[0]))
# for a ,b in zip(label,label_t):
#     if a!=int(b):
#         print(a,b)


In [24]:
pred=clf.predict(test_content.toarray())

In [25]:
print(type(pred))

<class 'numpy.ndarray'>


In [29]:
pred_list=[]
for value in pred:
    value=int(value)
    pred_list.append(value)

In [36]:
err_dict={}
with open('../data/test.csv','r') as f:
    re=csv.reader(f)
    next(re)
    re=list(re)
    for a, b in zip (pred_list,re):
        if int(a) != int(b[0]):
            err_dict[b[1]]=int(a)

In [37]:
print(len(err_dict))

109


In [39]:
for key in err_dict.keys():
    print(key,err_dict[key])
    print('\n')

亲爱的佳人们
 0


办年审 0


西城年华小四女及望月公寓小三女作业xx元/时；西溪风情初一女英科学xx元/时；之江公寓高一女数理化xx元/时请回电神舟家教xxxxxxxx佘
 0


请把房租打到招行xxxxxxxxxxxxxxxx邹晓燕
 0


xxxx开疆拓土!现房
 0


鼓楼龙江有房 0


知音半岛·泊翠 后官湖生态宜居新城内 0


安赢系列x年期保证年利率x.xx% 0


明月新城底商排号优惠政策～东升老业主排号五万抵十万～普通客户排号五万抵八万～有需要的可以和我联系～明月新城马小龙～ 0


健康在线尤维斯产品大促全场买二送一产品购一套即再送一瓶维e
 0


信用社xxxxxxxxxxxxxxxx佘远美
 0


本公司主要经营五金冲压及其制品 0


xxxxxxoxxxxxxxxxoxo农行张玉兰
 0


玛雅国际养生会所回馈新老顾客
 0


中国电信真xG、华为Matex免费送！预存xxxx元话费即送华为xG双模手机Matex 0


办各种证件电话xxxxxxxxxxx 0


本期水一木一金野兽开一特
 0


尊敬的晋商银行客户：x月xx日前发行理财
 0


开o/o發o/o剽xxxxixxxxxx王艳
 0


xxxxxxxxxxxxxxxxx 0


d今后不用麻烦去*奥*门了
 0


应城中百蒙牛奶粉全场惊爆x
 0


惊喜x[闪电][ 厂家特惠原价xxxx元面部护理年卡xx次现只需xxxx元xx次[炸弹][炸弹][炸弹]凡在开卡时间一年内做满xx次再 0


xxx期失利的友友
 0


单击服务＞办理＞业务办理＞自由组合套餐变更
 1


豁然文化学校在职一线金牌教师一对一全科辅导质量保证xx天提升xxx分精品小班师资力量雄厚签约教学价格全市最低提分热线xxxxxxxx 0


曾经往事常回首
 0


尊敬的贵宾：元宵佳节到
 0


急用钱 0


北方国内货运代 理处欢迎您 0


我厂为饭店快餐店制造安装改装维修节能炉冰柜空调电器清洗油烟机
 0


永乐莘庄龙之梦黑色星期五满额最高送x千积分全网比价xxx%正品保障活动x月xx日至xx日沪闵路xxxx号Bx咨询电话xxxxxxxx
 0


我是刚才给您打电话的大中华置业顾问【薛野】xxxxxxxxxxx 0


亲爱的VIP
 0


出的

In [40]:
from sklearn import metrics
def elevate_result(label,pred):
    """
    函数说明: 对分类器预测的结果进行评估，包括accurancy,precision,recall,F-score
    Parameter:
        label - 真实值
        pred - 预测值
    Return:
        None
    Modify:
        2017-12-22
    """
    con_mat = metrics.confusion_matrix(label,pred)
    TP = con_mat[1,1]
    TN = con_mat[0,0]
    FP = con_mat[0,1]
    FN = con_mat[1,0]
    
    accurancy = (TP+TN)/(TP+TN+FN+FP)
    precison = TP/(TP+FP) #精确率反应误报情况
    recall = TP/(TP+FN) # 召回率反应漏报情况
    beta = 1
    F_score = (1+pow(beta,2))*precison*recall/(pow(beta,2)*precison+recall)
    
    print("TP:",TP)
    print("TN:",TN)
    print("FP:",FP)
    print("FN:",FN) 
    print("accurancy: %s \nprecison: %s \nrecall: %s \nF-score: %s" % (accurancy,precison,recall,F_score))
    
    return [accurancy,precison,recall,F_score]


In [42]:
label_t=[]
with open('../data/test.csv','r') as f:
    re=csv.reader(f)
    next(re)
    for row in re:
        label_t.append(int(row[0]))
elevate_result(label_t,pred)

TP: 3861
TN: 36030
FP: 3
FN: 106
accurancy: 0.997275 
precison: 0.9992236024844721 
recall: 0.9732795563398033 
F-score: 0.9860809602860426


[0.997275, 0.9992236024844721, 0.9732795563398033, 0.9860809602860426]

In [ ]:
with open('../data/error_report.csv','w') as f:
    wr=csv.writer(f)
    wr.writerow(['pred','mess'])
    for key in err_dict:
        wr.writerow([err_dict[key],key])
    